# Enseignement d'intégration - Bitcoin Trading Bot - Groupe 7
## Bitcoin Forecasting using Deep Learning
### COLLEVILLE Tanguy, BOULAY Clément

![CentraleSupelec Logo](https://www.centralesupelec.fr/sites/all/themes/cs_theme/medias/common/images/intro/logo_nouveau.jpg)

In [ ]:
__author__ = "Colleville Tanguy", "Clément Boulay"
__version__ = "1.0.0"
__maintainer__ = "Colleville Tanguy"
__email__ = ["tanguy.colleville@student-cs.fr", "clement.boulay@student-cs.fr"]
__status__ = "Dev"

## 1.0 - Importation des librairies

In [ ]:
import os
import numpy as np
import time as time
import warnings



from matplotlib import pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,Bidirectional,Dropout,LSTM,BatchNormalization,Flatten,Conv1D
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import HeNormal,GlorotNormal
from scipy.stats import reciprocal
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import  StandardScaler



warnings.filterwarnings("ignore")
CURDIR = os.path.dirname(os.getcwd())
DATADIR = os.path.join(CURDIR,  "data")
%matplotlib inline

In [ ]:
#paramètres de représentation
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

norme : pep8

## 1.1 - Importation des données
Les données proviennent du site https://blockchain.info/.

In [ ]:
df_blockchain = pd.read_csv(os.path.join(DATADIR, "df_blockchain.csv"), delimiter=",")

## 2.1 - Découverte des données
Les questions auxquelles on cherche à répondre ici sont :
- Quelles sont les variables explicatives, que représentent-elles ? Qui est la variable à expliquer, que représente-t-elle ?
- De quelle forme est le jeu de données (types des variables, shape, complet, incomplet) ?
- Peut-on extraire une corrélation entre certaines variables ? Lesquelles choisir pour la modélisation ?

In [ ]:
df_blockchain.head()## affichage 5 premières lignes

In [ ]:
df_blockchain.columns.sort_values().shape

In [ ]:
df_blockchain.describe()

In [ ]:
print(df_blockchain.info(memory_usage='deep'))## donne le vrai usage de mémoire du df 
print(df_blockchain.memory_usage(deep=True))## stockage par série

In [ ]:
print(df_blockchain.isna().sum())

In [ ]:
print(df_blockchain.shape)
df_blockchain.dropna(axis=0,inplace=True)## drop les lignes où y'a des nan 
print(df_blockchain.shape)

In [ ]:
df_blockchain.dtypes.value_counts().plot.pie(title="features' type")# petit camenbert des différents type du dataframe
plt.xlabel(" ")
plt.ylabel(" ")
plt.show()

Signification des variables (d'après https://blockchain.info/) :
![notebook\features.png](attachment:notebook\features.png)


In [ ]:
for col in df_blockchain.select_dtypes('float'):
    plt.title(f"Distribution of {col}")
    sns.distplot(df_blockchain[col])
    plt.grid()
    plt.show()

On observe que certaines distributions sont piquées (trade-volume, transaction-fees, estimated-transaction-volume, output_volume) alors que d'autres sont étalées (avg-block-size, n-transactions-per-block). Pour la distribution du market-price, on observe un fort pic autour de 0, qui correspond aux premières années du Bitcoin (quand il était à moins de 100$). On retrouve ce pic caractéristique autour de 0 dans plusieurs autres distributions (trade-volume, miners-revenue, difficulty, hash-rate), avec toujours la même signification temporelle. Enfin, pour certaines distributions, on peut clairement voir une évolution temporelle ("de gauche à droite") : c'est le cas pour total-bitcoins (valeurs présentes à droite du graphe) et difficulty.

In [ ]:
for col in df_blockchain.select_dtypes('object'):
    print(col)

In [ ]:
Price=df_blockchain["market-price"]

In [ ]:
plt.plot(Price.values)
plt.title("Price of BTC evolution")
plt.ylabel("BTC price, $ USD" )
plt.xlabel("days")
plt.grid()

## 2.2 - Exploration des liens entre les données

In [ ]:
plt.plot(df_blockchain["total-bitcoins"]/500,label="number of BTC mined ")
plt.plot(df_blockchain["market-price"],label="price")
plt.title("Number of BTC mined and price evolution over time")
plt.xlabel("days")
plt.ylabel("USD $ and number of BTC/")
plt.legend()
plt.grid()

In [ ]:
plt.plot(Price[1750:-30],label="what we have for training")
plt.plot(Price[-30:],label="What we want to predict")
plt.title("Price of BTC evolution")
plt.ylabel("BTC price, $ USD" )
plt.xlabel("days")
plt.legend()
plt.grid()

On peut penser que la variation brusque du cours du Bitcoin sur la période de test va poser des problèmes dans la mesure où cette chute de prix n'a pas eu lieu par le passé donc est imprévisible.

In [ ]:
df_indexed=df_blockchain.set_index("Date")

In [ ]:
df_indexed.index = pd.to_datetime(df_blockchain.set_index("Date").index)
min_year=df_indexed.index.year.min()
max_year=df_indexed.index.year.max()
print("min year", df_indexed.index.year.min())
print("max year",df_indexed.index.year.max())

In [ ]:
df_indexed.index = pd.to_datetime(df_blockchain.set_index("Date").index)
df_blockchain["Year"]=df_indexed.index.year
df_blockchain["Month"]=df_indexed.index.month
df_blockchain["Day"]=df_indexed.index.day
df_blockchain["week_day"]=df_indexed.index.weekday

In [ ]:
colormap=plt.cm.gist_ncar
colost=[colormap(i) for i in np.linspace(0,0.95,1+max_year-min_year)]## make a nice colorbar

In [ ]:
plt.figure()
plt.title("Market price over year")
for i in range(min_year,max_year+1,1):
    plt.plot(df_blockchain[df_blockchain["Year"]==i]["market-price"].values,color=colost[i-min_year],label=i)
plt.legend()
plt.xlabel("days")
plt.ylabel("USD $")
plt.grid()

On ne voit pas de saisonnalité dans les prix même si on peut remarquer un engoument certain pour le btc ces dernières années. 

In [ ]:
df_blockchain.columns

In [ ]:
for j in df_blockchain.columns[2:-4]:
    plt.figure()
    for i in range(min_year,max_year+1,1):
        plt.title(f"{j} over year")
        plt.plot(df_blockchain[df_blockchain["Year"]==i][j].values,color=colost[i-min_year],label=i)
        plt.xlabel("days")
        plt.ylabel(j)
        plt.legend()
    plt.grid()
    plt.show()

Cette série de graphes montre l'évolution de chacune des variables au travers des années. Certaines représentations sont particulièrement intéressantes. Par exemple, on observe que le hash-rate augmente de façon cosntante depuis 2016. C'est également le cas pour la difficulté et la blocks-size (qui augmente linéairement, ce qui est logique puisque la taille de la blockchain ne fait que croître). D'autre part, certains graphes indiquent que l'année 2021 est particulière pour le marché du Bitcoin : le cost-per-transaction est par exemple plus élevé cette année qu'auparavant (facteur 2.5 au plus haut !). De la même façon, ces graphes permettent d'identifier des moments marquants de l'histoire du Bitcoin. À ce titre, on identifie un fort volume de transaction en février-mars 2011 (en Bitcoin, cela n'apparaît pas en USD car à cette époque le BTC ne valait rien, c'est le moment de la parité avec l'USD), ainsi que depuis 2020 dans l'échelle USD (qui ne traduit pas forcément plus de transactions mais traduit un prix qui explose). Enfin, on observe une explosion de la capitalisation "market-cap" depuis deux ans, qui montre l'attrait pour les cryptomonnaies. D'une façon générale, on ne remarque pas de saisonnalité dans les données.

In [ ]:
target="market-price"
for i in df_blockchain.select_dtypes('float'): 
    print(i)
    plt.figure()
    plt.title(f"{i} and {target}")
    plt.scatter(df_blockchain[i],df_blockchain[target])
    plt.xlabel(i)
    plt.ylabel(target)
    plt.grid()
    plt.show()

La série de graphes suivante montre les variations de chacune des 22 variables explicatives en fonction de la variable à expliquer "market-price". De façon notable, on remarque une corrélation entre cette dernière et les variables suivantes : miners-revenue, transaction-fees-usd, cost-per-transaction, estimated-transaction-volume-usd, market-cap (de façon notable).
De plus, pour les variables explicatives dont le comportement est approximativement linéaire du temps (difficulty, hash-rate au début, blocks-size, n-transactions-total), on retrouve la forme du graphe des prix représenté au-dessus (cela revient indirectement à représenter le prix en fonction du temps).

## 2.3 - Analyse des corrélations

In [ ]:
plt.figure(figsize=(16,7))
plt.title("Correlation heatmap")
sns.heatmap(df_blockchain.corr(),cmap="seismic",annot=True)

In [ ]:
corr_matrix=df_blockchain.corr()
sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).stack().sort_values(ascending=False))
p=0
coupled=[]
for index, value in sol.items():
    if value>0.9:
        p+=1
        coupled.append(index)
print(p,"features have more than 90% of correlation")

In [ ]:
print(coupled)

### plusieurs approches sont possibles : nous les mettrons en oeuvre par la suite lors de notre étape de preprocessing ici des tris sont fait simplement pour voir si la suppresion de variables "parasites" révèlent des tendances ou des relations "cachées"
* la première serait de supprimer les variables corrélées entre elles dans la mesure où l'une explique l'autre et d'enlever les variables fortement corrélée avec la target surtout celle qui dérive de market-price. 
* la deuxième est d'utiliser un random forest pour mesurer l'importance des features ou encore utiliser une regression LASSO 
* la troisième est d'étudier les valeurs propres de la matrice de corrélation pour trier. En effet, de faibles valeurs propres indiques que des features sont redondants car la famille devient liée on peut ainsi évincer certains features. 

In [ ]:
df_blockchain.columns

In [ ]:
features_to_be_dropped=["market-cap","difficulty","n-unique-addresses","n-transactions-excluding-popular","estimated-transaction-volume-usd"]
df_blockchain.drop(features_to_be_dropped,axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(16,7))
plt.title("Correlation heatmap with less features")
sns.heatmap(df_blockchain.corr(),cmap="seismic",annot=True)

In [ ]:
plt.figure()
for i in range(0,7,1):
    plt.title(f"market price over week day ")
    plt.plot(df_blockchain[df_blockchain["week_day"]==i]["market-price"].values,color=colost[i],label=i,alpha=0.7)  
plt.legend()
plt.ylabel("$ USD")
plt.xlabel("Days")
plt.grid()
plt.show()

Visiblement il n'y pas tant de traders du "dimanche". En effet, au vu de l'engouement des particuliers pour cette crypto nous aurions pu nous attendre à voir des tendances le week end par exemple mais ce n'est pas le cas. NB :  0 Lundi, 1 Mardi etc.. 

In [ ]:
colormap=plt.cm.gist_ncar
colostm=[colormap(i) for i in np.linspace(0,0.95,1+12)]## len du nb de trucs à plot

plt.figure()
for i in range(0,13,1):
    plt.title(f"market price over month ")
    plt.plot(df_blockchain[df_blockchain["Month"]==i]["market-price"].values,label=i,color=colostm[i])
plt.legend()
plt.ylabel("BTC price $ USD")
plt.xlabel("days")
plt.grid()
plt.show()

Les mois non plus ne sont dégages pas de tendances sur le prix. Ceci vient confirmer la correlation matrix illustrée par la heatmap dans laquelle on voit que les correlations sont faibles pour les jours de la semaine et les mois. 

## 3.1 - Preprocessing 

Le "preprocessing" (en français, préparation préliminaire) est une méthode utilisée en Machine Learning dans le but de mettre en forme les données afin qu'elles correspondent au format d'entrée des modèles (par ex. réseaux de neurones). Cette phase de préparation est essentielle et permet d'optimiser l'efficacité des algorithmes employés ensuite. 

Le Rasoir d'Ockham est un principe de raisonnement philosophique entrant dans les concepts de rationalisme et de nominalisme. Il peut se comprendre par la locution suivante "les multiples ne doivent pas être utilisés sans nécessité". C'est un principe phare du machine learning dans la mesure où nous cherchons toujours les modèles les plus rapides et les plus parcimonieux pour un même résultat. 

In [ ]:
def imputation(df):
    """
    Entries : 
        df (pandas DataFrame) : tableau contenant les données brutes (présence de NaN)
    ====================================================================================================
    Aim : 
        Retirer les valeurs NaN des données du tableau
    ====================================================================================================
    Outputs: 
        df (pandas DataFrame) : tableau de données nettoyé des valeurs NaN
    """
    df.dropna(axis=0,inplace=True)
    return(df)

In [ ]:
def encodage(df):
    # rien ici car le jeu de données est déjà au bon format (float64)
    return df 

In [ ]:
def keep_relevant_features(df,features):

    """
    Entries :
    df (pandas DataFrame) : tableau contenant toutes les variables explicatives
    features (list) : variables explicatives à garder (celles peu corrélées entre elles)
    ====================================================================================================
    Aim :
    Identifier toutes les variables explicatives redondantes (corrélées entre elles) du tableau, qui n'apportent pas d'information
    ====================================================================================================
    Outputs:
    my_features (list) : variables explicatives à garder pour l'étude
    """

    myfeatures=features
    correlation=df.corr()

    colonnes=df.columns.drop("Date").drop("market-price")

    for ligne in colonnes:

        for col in colonnes:

            if 0.9<correlation[ligne][col]<1:

                if col in myfeatures:

                    myfeatures=myfeatures.drop(col)

            elif correlation[ligne][col]==1:
                break
    return myfeatures

In [ ]:
def feature_engineering(df):  
    """
    Entries :
    df (pandas.DataFrame) : tableau de données complet
    ====================================================================================================
    Aim :
    Créer de nouvelles variables explicatives et retirer celles trop corrélées
    ====================================================================================================
    Outputs :
    df (pandas.Dataframe) : tableau de données avec de nouvelles variables apportant de l'information
    """
    ########################################################## DATE ##################################################################
    features_to_be_dropped=["market-cap","difficulty","n-unique-addresses","n-transactions-excluding-popular","estimated-transaction-volume-usd","n-transactions-total"]# variables à éliminer selon une analyse visuelle de la matrice de corrélation
    relevant_feature=keep_relevant_features(df,df.columns)
    for col in df.columns:
        if col not in relevant_feature and col!="market-price" and col!="Date": # on élimine les colonnes identifiées
            df=df.drop(columns=[col])
    for col in features_to_be_dropped : # on vérifie a-posteriori que le tri s'est bien déroulé
        if col in df.columns : 
            df=df.drop(columns=[col])

    # création de nouvelles variables à partir de la date
    df_indexed=df.set_index("Date")
    df_indexed.index = pd.to_datetime(df.set_index("Date").index)
    df["Year"]=df_indexed.index.year
    df["Month"]=df_indexed.index.month
    df["Day"]=df_indexed.index.day
    df["week_day"]=df_indexed.index.weekday
    df.index = pd.to_datetime(df.set_index("Date").index)# on retire la date car l'information est désormais stockée
    df.drop("Date",axis=1,inplace=True)
    ## l'étude de l'importance des features grâce à random forest [dans l'autre notebook] a confirmé la selection des features déjà réalisée
    # avec plus de temps : 
    ## à partir de regression on aurait pu créer d'autres features : combinaison de features 
    ## on aurait pu également faire des moyennes glissantes sur des plages de temps variées (fenêtrage)
    return(df)

In [ ]:
def scaler(df):
    """
    Entries : 
        df (pandas DataFrame) : tableau contenant les données brutes (non-standardisées)
    ====================================================================================================
    Aim : 
        Retourner un tableau de données standardisé : pour chaque variable explicative,
        moyenne nulle et écart-type égal à 1.
    
    Outputs: 
        df (pandas DataFrame) : tableau de données standardisé
    """
    scale = StandardScaler()
    market_price = df.pop("market-price")
    market_price = market_price.reset_index(drop=True)
    columns = df.columns
    df = scale.fit_transform(df)
    df = pd.DataFrame(df, columns = columns)
    df.insert(0, "market-price", market_price)
    return df

In [ ]:
def preprocessing(df):
    """
    Entries : 
        df (pandas DataFrame) : tableau contenant les données brutes
    ====================================================================================================
    Aim : 
        Retourner un tableau de données prêt à l'emploi (standardisé, sans NaN, avec des variables explicatives
        sans grande corrélation)
    ====================================================================================================
    Outputs: 
        X -->pandas Dataframe df sans market price
        Y -->pandas serie market price 
        df (pandas DataFrame) : tableau de données prêt à l'emploi
    """
    df = imputation(df)
    df = encodage(df)
    df = feature_engineering(df)
    df = scaler(df)
    X = df.drop('market-price', axis=1)
    y = df["market-price"]
    return X, y,df

## 3.2 - Mise en forme des données pour le réseau LSTM

Here we split and process data before training.

LSTM layer as an input layer expects the data to be 3 dimensions, we will use 'process_data' function to split data into sequences of a fixed length (rnn_size).

The neural network is expecting to have an input's shap of [batch_size, rnn_size, nb_features]

In [ ]:
df_blockchain = pd.read_csv(os.path.join(DATADIR, "df_blockchain.csv"), delimiter=",")

In [ ]:
X,y,df=preprocessing(df_blockchain[2000:])

In [ ]:
def splitting(df,start,nb_split,size_test):
    """
    Entries : 
        df (pandas DataFrame)
        start (int) : jour de début de l'étude
        nb_split (int) : represente le nombre de découpe pour la cross validation 
        size_test (int) : représente le nombre de jour du test set 
    ====================================================================================================
    Aim : Créer une liste de X,y pour réaliser une cross validation avec le respect de la temporalité
    ====================================================================================================
    Outputs: 
        list_df_train --> list of df sampled train 
        list_df_test --> list of df sampled test
    """

    tscv = TimeSeriesSplit(n_splits=nb_split, test_size=size_test)
    list_df_train,list_df_test=[],[]
    df=df.iloc[start:]
    for train_index, test_index in tscv.split(df):
        print("TRAIN:", train_index, "TEST:", test_index)
        df_train=df.iloc[train_index[0]:train_index[-1]]
        df_test=df.iloc[test_index[0]:test_index[-1]]
        list_df_test.append(df_test)
        list_df_train.append(df_train)
    return list_df_train,list_df_test

In [ ]:
columns=df.columns
columns

In [ ]:
def process_data(data, rnn_size=3, target_id=0, columns_size=len(columns)-1):## -1 car on ne veut pas market price dans X 
    """
    Entries:
    data Pandas DataFrame to be processed for RNN 
    rnn_size (int) --> nombre de récurrence d'étude pour le RNN 
    target_id (int) --> nombre de la colonne objectif 

    ====================================================================================================
    Aim: processing X et y for RNN 
    ====================================================================================================
    Outputs:
    X_train (np.array)
    y_train (np.array)
    """
    X = []
    y = []
    for i in range(len(data)-rnn_size):
        X.append(data.iloc[i:i+rnn_size,1:])# pas de market price dans X 
        y.append(data.iloc[i+rnn_size,0])
    return np.array(X).astype(np.float32).reshape((-1,rnn_size,columns_size)), np.array(y).astype(np.float32)

In [ ]:
def split_test_train(list_df_train, list_df_test,rnn_size=3):
    """
    Entries:
    list_df_train (list) : liste contenant les DataFrame du jeu d'entraînement de cross-validation
    list_df_test (list) : liste contenant les DataFrame du jeu de test de cross-validation
    ====================================================================================================
    Aim: Génère 4 listes, une contenant les variables explicatives d'entraînement, une contenant la variable à expliquer d'entraînement et pareil pour le test
    ====================================================================================================
    Outputs:
    list_X_train (list)
    list_X_test (list)
    list_y_train (list)
    list_y_test (list)
    """
    list_X_train, list_X_test, list_y_train, list_y_test = [], [], [], []
    for df_train in list_df_train:
        X_train, y_train = process_data(df_train,rnn_size=rnn_size)
        list_X_train.append(X_train)
        list_y_train.append(y_train)
    for df_test in list_df_test:
        X_test, y_test = process_data(df_test,rnn_size=rnn_size)
        list_X_test.append(X_test)
        list_y_test.append(y_test)
    return list_X_train, list_X_test, list_y_train, list_y_test

In [ ]:
list_df_train,list_df_test=splitting(df,0,4,30)

In [ ]:
look_back=20
list_X_train, list_X_test, list_y_train, list_y_test=split_test_train(list_df_train, list_df_test,rnn_size=look_back)

Nous créerons un list_X_val et list_y_val par la suite 
* **

## 4.1 - Régressions linéaire classique et régression Lasso

 Lasso and Linear Regression to see if simple method perform well and the importance of features which has been compared with random forest features importance estimation

### 4.1.1 - Approche naïve
Comme suggéré durant la phase de questions de la présentation initiale (mercredi), nous avons mis en place une approche naïve qui consiste à prédire la valeur du Bitcoin au jour J en utilisant la moyenne glissante des 20 journées précédentes. Cette implémentation nous permettre ensuite de comparer cette approche archaïque aux modèles plus élaborés.

In [ ]:
# définition des jeux d'entraînement, de validation et de test
X_train = X.iloc[:1400, :].values
X_test = X.iloc[1401:, :].values
y_train = y[:1400].values
y_test = y[1401:].values

In [ ]:
plt.plot(np.linspace(1000, 1400, 400), y_train[-400:], "r--")
plt.plot(np.linspace(1401, 1540, 139), y_test, "m--")
plt.xlabel("Jours")
plt.ylabel("Prix (USD)")
plt.grid()
plt.title("Prix de la période d'entraînement (rouge) et de test (violet)")

In [ ]:
print('X_train shape : {}'.format(X_train.shape))
print('y_train shape : {}'.format(y_train.shape))
print('X_test shape : {}'.format(X_test.shape))
print('y_test shape : {}'.format(y_test.shape))

In [ ]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]

In [ ]:
# standardisation des données
scaler = StandardScaler().fit(X_train)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

In [ ]:
# méthode naïve : on calcule la moyenne des prix des 20 jours précédents pour prédire le prix au jour J
prices = df["market-price"]
index = 0
pred_prices = []
for new_price in y_test:
    if index == 0:
        pred_price = y_train[-20 + index:].mean()
    else:
        pred_price = np.concatenate((y_train[-20 + index:], y_test[0:index]), axis = 0).mean()
    pred_prices.append(pred_price)
    index +=1

In [ ]:
MSE = 0
N = len(y_test)
for val in range(N):
    MSE += 1/N * (pred_prices[val] - y_test[val])**2
RMSE = round(np.sqrt(MSE), 2)
print("La valeur RMSE de la prédiction naïve est de", RMSE, "$")

Le score est mauvais, on en déduit qu'un modèle plus raffiné est nécessaire pour obtenir une bonne prédiction du prix du Bitcoin.
### 4.1.2 - Régression linéaire classique

Les variables explicatives (features) sont déjà sélectionnées en fonction de leurs caractériques (elles sont passées par la fonction "preprocessing").

In [ ]:
# construction et fit du modèle
lin_reg = LinearRegression().fit(X_train_n, y_train)

# prédiction 
y_train_hat = lin_reg.predict(X_train_n)
y_test_hat = lin_reg.predict(X_test_n)

# erreur d'entraînement et de test
err_train = mean_squared_error(y_train_hat, y_train, squared = False) # valeur RMSE
err_test = mean_squared_error(y_test_hat, y_test, squared = False)
print("L'erreur d'entraînement est de {:.2f}$ et celle de test est de {:.2f}$.".format(err_train, err_test))

In [ ]:
plt.figure(figsize = (14, 4))
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_hat, label = 'Prédictions vs. réel (jeu de test)')
plt.plot([min(y_test),max(y_test)], [min(y_test), max(y_test)], "r--")
plt.legend()
plt.title("Régression linéaire classique (jeu de test)")
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_hat, label = "Prédictions vs. réel (jeu d'entraînement)")
plt.plot([min(y_train),max(y_train)], [min(y_train), max(y_train)], "r--") # ligne rouge pointillée représente la prédiction parfaite
plt.legend()
plt.title("Régression linéaire classique (jeu d'entraînement)")

Malgré la feature engineering, le score n'est pas très bon. Peut-on faire mieux avec une régression Lasso ? LA regression va nous informer sur l'importance des paramètres parmis ceux retenus, nous étudirions également par la suite si Lasso sur la dataset brut nous conduit à la même selection de modèle. 

### 4.1.3 - Regression Lasso sur les données pré-processées

In [ ]:
param_grid = {"alpha": np.logspace(0, 2, 20)}
lasso = Lasso()
cv = GridSearchCV(lasso, param_grid = param_grid, cv = 3) # cross-validation
cv.fit(X_train_n, y_train)

# représentation des coefficients du meilleur estimateur
print("Nombre de coefficients non-nuls :", (cv.best_estimator_.coef_ != 0).sum())
print("Meilleure valeur pour alpha :", cv.best_params_['alpha'])

fig, ax = plt.subplots()
ax.plot(cv.best_estimator_.coef_, "rx")
ax.set_title('Coefficients de la régression Lasso')
plt.xlabel("Variables explicatives")
plt.ylabel("Coefficients")

On peut donc voir que les 3 variables explicatives les plus importantes, sont  la 7, la 11 et la 12 (respectivement "cost-per-transaction", "year" et "month").

In [ ]:
err_train_lasso = mean_squared_error(cv.best_estimator_.predict(X_train_n), y_train, squared = False)
err_test_lasso = mean_squared_error(cv.best_estimator_.predict(X_test_n), y_test, squared = False)
y_pred = cv.best_estimator_.predict(X_test_n)
y_train_pred = cv.best_estimator_.predict(X_train_n)

print("L'erreur d'entraînement est de {:.2f}$ et celle de test est de {:.2f}$.".format(err_train_lasso, err_test_lasso))

plt.figure(figsize = (14, 4))
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, label = 'Prédictions vs. réel (jeu de test)')
plt.plot([min(y_test),max(y_test)], [min(y_test), max(y_test)], "r--")
plt.legend()
plt.title("Régression linéaire du Lasso sur le jeu de test")
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, label = "Prédictions vs. réel (jeu d'entraînement)")
plt.plot([min(y_train),max(y_train)], [min(y_train), max(y_train)], "r--") # ligne rouge pointillée représente la prédiction parfaite
plt.legend()
plt.title("Régression linéaire classique (jeu d'entraînement)")

Une explication possible est la rupture de tendance observée sur le graphe du prix en USD du Bitcoin plus haut (graphe bicolore). En effet, si on représente tous les prix sur un même graphe, on remarque une hétérogénéité de leur distribution :

In [ ]:
plt.figure(figsize = (14, 4))
plt.scatter(y_test, y_pred, label = 'Prédictions vs. réel (jeu de test)')
plt.scatter(y_train, y_train_pred, label = "Prédictions vs. réel (jeu d'entraînement)")
plt.plot([min(y_train),max(y_train)], [min(y_train), max(y_train)], "g--") # ligne rouge pointillée représente la prédiction parfaite
plt.plot([min(y_test),max(y_test)], [min(y_test), max(y_test)], "r--")
plt.legend()
plt.grid()
plt.title("Régression linéaire du Lasso")

### 4.1.4 - Régression Lasso sur les données brutes (directement depuis le .csv)

Le modèle Lasso possède un hyperparamètre $\alpha$ qu'il faut fixer. Pour trouver la meilleure valeur possible pour $\alpha$, on effectue une validation croisée sur le jeu d'entraînement en utilisant l'objet GridSearchCV de sklearn.

In [ ]:
df_blockchain.dropna(axis=0,inplace=True)
X=df_blockchain.drop(["market-price","Date"],axis=1)
y=df_blockchain["market-price"]
X_train = X.iloc[:1400, :].values
X_test = X.iloc[1401:, :].values
y_train = y[:1400].values
y_test = y[1401:].values
scaler = StandardScaler().fit(X_train)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

In [ ]:
param_grid = {"alpha": np.logspace(0, 2, 20)}
lasso = Lasso()
cv = GridSearchCV(lasso, param_grid = param_grid, cv = 3) # cross-validation
cv.fit(X_train_n, y_train)

# représentation des coefficients du meilleur estimateur
print("Nombre de coefficients non-nuls :", (cv.best_estimator_.coef_ != 0).sum())
print("Meilleure valeur pour alpha :", cv.best_params_['alpha'])



In [ ]:
fig, ax = plt.subplots()
ax.plot(cv.best_estimator_.coef_,"rx")
ax.set_title('Coefficients de la régression Lasso')
plt.xlabel("Variables explicatives")
plt.ylabel("Coefficients")
plt.grid()

Ceci vient confirmer notre étude passée. 

In [ ]:
err_train_lasso = mean_squared_error(cv.best_estimator_.predict(X_train_n), y_train, squared = False)
err_test_lasso = mean_squared_error(cv.best_estimator_.predict(X_test_n), y_test, squared = False)
y_pred = cv.best_estimator_.predict(X_test_n)
y_train_pred = cv.best_estimator_.predict(X_train_n)

print("L'erreur d'entraînement est de {:.2f}$ et celle de test est de {:.2f}$.".format(err_train_lasso, err_test_lasso))

plt.figure(figsize = (14, 4))
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, label = 'Prédictions vs. réel (jeu de test)')
plt.plot([min(y_test),max(y_test)], [min(y_test), max(y_test)], "r--")
plt.legend()
plt.title("Régression linéaire du Lasso sur le jeu de test")
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, label = "Prédictions vs. réel (jeu d'entraînement)")
plt.plot([min(y_train),max(y_train)], [min(y_train), max(y_train)], "r--") # ligne rouge pointillée représente la prédiction parfaite
plt.legend()
plt.title("Régression linéaire classique (jeu d'entraînement)")

Une explication possible est la rupture de tendance observée sur le graphe du prix en USD du Bitcoin plus haut (graphe bicolore). En effet, si on représente tous les prix sur un même graphe, on remarque une hétérogénéité de leur distribution :

In [ ]:
plt.figure(figsize = (14, 4))
plt.scatter(y_test, y_pred, label = 'Prédictions vs. réel (jeu de test)')
plt.scatter(y_train, y_train_pred, label = "Prédictions vs. réel (jeu d'entraînement)")
plt.plot([min(y_train),max(y_train)], [min(y_train), max(y_train)], "g--") # ligne rouge pointillée représente la prédiction parfaite
plt.plot([min(y_test),max(y_test)], [min(y_test), max(y_test)], "r--")
plt.legend()
plt.grid()
plt.title("Régression linéaire du Lasso")

## 5.1 - Modèles de Deep Learning (réseaux de neurones) pour la prédiction

Aim : to predict the 30 last day of BTC
* **

Recall of Deep learning in a noteshell : 
* **Metric** values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.


* A **loss function** is used to train your model. A metric is used to evaluate your model. A loss function is used during the learning process. A metric is used after the learning process. 


* An **optimizer** is a method or algorithm to update the various parameters that can reduce the loss in much less effort. 


* The **learning rate** is a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. This is a key parameters. We have to take the half of the learning that makes the model diverging. In other word, we have te start with a relativly big learning rate and decrease manually by dividing by 2 for example, till it stop to diverge.

* **Drop** : dropout refers to ignoring units (i.e. neurons) during the training phase of certain set of neurons which is chosen at random. It aims to reduce risk of overfitting. 


* **Epochs** : indicates the number of passes of the entire training dataset the machine learning algorithm has completed


* **Early stopping** : Early stopping is designed to monitor the generalization error of one model and stop training when generalization error begins to degrade. In a way it means you will not do epochs for nothing. 


* **Batch size** :  refers to the number of training examples utilized in one iteration. Small batch size increase risk of overfitting because the weights are computed for each batch. Therefore there is a comprise to do. 


* **Verbose** : degree of detail when executing a model


* **RNN** : Recurrent Neural Network, are a class of neural networks that allow previous outputs to be used as inputs while having hidden states.


* **CNN** : Convolutional Neural Network 

* **Batch normalization** : In a 2015 paper, Sergey Ioffe and Christian Szegedy proposed a technique called Batch Normalization (BN) to address the vanishing/exploding gradients problems. The technique consists of adding an operation in the model just before or after the activation function of each hidden layer, simply zero-centering and normalizing each input, then scaling and shifting the result using two new parameter vectors per layer: one for scaling, the other for shifting. In other words, this operation lets the model learn the optimal scale and mean of each of the layer’s inputs. 

* **Initializer** :  define the way to set the initial random weights of Keras layers. It is a way to avoid vanishing or exploding gradient. There are many intializer but they have to be choose accordingly to the activation function : 
        Glorot uniform or normal for : None, Tanh, Logistic, Softmax
        He uniform or normal for : ReLU & variants

* **

*  **training dataset**  is a set of examples used to fit the parameters (e.g. weights of connections between neurons in artificial neural networks) of the model.
* ** test dataset ** is a dataset used to provide an unbiased evaluation of a final model fit on the training dataset.If the data in the test dataset has never been used in training (for example in cross-validation), the test dataset is also called a holdout dataset. The term "validation set" is sometimes used instead of "test set" in some literature


* **

## RNN 
| Advantages | Drawbacks |
|:-:|:-:|
| Model size not increasing with size of input | Computation being slow |
| Computation takes into account historical information| Difficulty of accessing information from a long time ago |
| Weights are shared across time | Cannot consider any future input for the current state|
| Possibility of processing input of any length |

Therefore we can easly understand why the RNN are very good for time series forecasting. 

Gated Recurrent Unit (GRU) and Long Short-Term Memory units (LSTM) deal with the vanishing gradient problem encountered by traditional RNNs, with LSTM being a generalization of GRU. 
* **
### 1. LSTM standing for Long Short Term Memory (RNN type):

Bi-directional LSTM is a variant : 
LSTM in its core, preserves information from inputs that has already passed through it using the hidden state.

Unidirectional LSTM only preserves information of the past because the only inputs it has seen are from the past.

Using bidirectional will run your inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backwards you preserve information from the future and using the two hidden states combined you are able in any point in time to preserve information from both past and future.

We tried out this kind of RNN (BidirLSTM) but after few reflexions it was not a good idea because of temporality importance. 
* **

### 2. Classical ANN : 
testing for many differents architecture : We have to be careful about the number of parameters because we only have a time series with 1500 days and about 16 columns, which is very few compared to what is used to be in DL. Therefore, we have limited our number of hidden layers and the number of neurons per layer. 

* To determine the number of layers : increase gradually the number of layer till overfitting. 


* The number of neurons per layers is determine by the problem here the number of data is limited. Moreover a "cone" architechture is more commune.

* To determine the learning rate : We have to take the half of the learning that makes the model diverging. In other word, we have te start with a relativly big learning rate and decrease manually by dividing by 2 for example, till it stop to diverge. 

* To determine batch_size and optimizer we have done some benchmark to see what was the best configuration. 
* **

### 3. CNN + LSTM : 
Here, we have stacked one layer of 1D convolution, 1 layer of LSTM and one Dense. 
Clearly this ANN is more complicated than the other two, it requires much more epochs to be trained even if there is the amount of parameters. 
Concerning hyperparameters we have choosen a random search to fine tune our model. 
* **

More generally we always started with very simple model and then we have augmented the complexity to get better results. To be confident in our choices, we have to keep in mind that we only have 1500 datas with 16 features, so the number of parameters to be optimized must not be to much large. 

Firstly let's define what a **GridSearchCV** is. This is an Exhaustive search over specified parameter values for an estimator. GridSearchCV implements a “fit” and a “score” method. It also implements “score_samples”, “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid. 
We have used **Randomized Search** for time efficiency. In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.

* **
## Optimizer : 
* **RMSprop** : The gist of RMSprop is to:Maintain a moving (discounted) average of the square of gradients Divide the gradient by the root of this average This implementation of RMSprop uses plain momentum, not Nesterov momentum. The centered version additionally maintains a moving average of the gradients, and uses that average to estimate the variance. In other words, it reduce the moving in one direction to prefer the one which will hep to get a faster convergence.
* **Adam** : Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments. According to Kingma et al., 2014, the method is "computationally efficient, has little memory requirement, invariant to diagonal rescaling of gradients, and is well suited for problems that are large in terms of data/parameters".
There are so many other Optimizer available with keras.

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    """
    Input:
    y_true (list or numpy.Array) : vraies valeurs du prix du Bitcoin sur la fenêtre temporelle de test
    y_pred (list or numyp.Array) : valeurs de prix prédites par l'algorithme
    Aim:
    Retourne la racine de l'erreur quadratique moyenne, une métrique permettant de juger de la performance de l'algorithme
    Output:
    rmse (float) : racine de l'erreur quadratique moyenne
    """
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
def create_model_LSTM(opti="RMSprop",ls=35,Den=10,learning_r=0.02):

    """
    Input:
    opti (string) : optimiseur à considérer pour l'entraînement du réseau de neurones. Les valeurs supportées sont : "RMSprop" et "Adam".
    ls (int) : nombre de neurones dans la couche LSTM
    Den (int) : nombre de neurones dans la couche cachée Dense
    learning_r (float) : vitesse d'apprentissage du réseau

    Aim:
    Créer un modèle de réseau de neurones paramétrables par l'optimiseur, le nombre de neurones des couches principales et la vitesse d'apprentissage.

    Output:
    model (keras.models) : modèle LSTM à une couche dense cachée et une couche Dropout
    """
    
    if not isinstance(opti, str):
            raise Exception(
                " opti must be a string which represents optimizer : Adam or RMSprop")
    if not isinstance(ls, int):
            raise Exception(
                " ls must be an int which represents number of unites for LSTM")
    if not isinstance(Den, int):
            raise Exception(
                " Den must be an int which represents number of neurons of dense")

    drop=0.3
    initHe= HeNormal()
    initGlo= GlorotNormal()
    model = keras.Sequential()
    bias=True
    model.add(LSTM(ls,input_shape=(look_back, len(columns)-1),dropout=drop,kernel_initializer=initGlo))
    # model.add(BatchNormalization())
    model.add(Dropout(drop))
    model.add(Dense(Den,use_bias=bias))
    # model.add(BatchNormalization())
    model.add(Dropout(drop))
    model.add(Dense(1))

    if opti=="RMSprop":
        opt = keras.optimizers.RMSprop(learning_rate=learning_r,
        rho=0.9,
        momentum=0.0,
        epsilon=1e-07,
        centered=False,
        name="RMSprop")## centered peut-être à mettre en True 
        
    if opti=="Adam":
        opt =keras.optimizers.Adam(
        learning_rate=learing_r,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam")

    model.compile(optimizer = opt, loss = root_mean_squared_error,metrics = [keras.metrics.RootMeanSquaredError()])
    model.summary()

    return model

In [ ]:
def create_model_CNN_RNN(opti="RSMprop",filtre=3,kernel=3,ls=5,learning_r=0.15):

    """
    Input:
    opti (string) : optimiseur à considérer pour l'entraînement du réseau de neurones. Les valeurs supportées sont : "RMSprop" et "Adam".
    filtre (int) : dimension de l'espace de sortie
    kernel (int) : dimension du noyau de convolution 1D
    ls (int) : nombre de neurones dans la couche LSTM
    learning_r (float) : vitesse d'apprentissage du réseau

    Aim:
    Créer un modèle de réseau de neurones paramétrables par l'optimiseur, le nombre de neurones des couches LSTM et Dense, la vitesse d'apprentissage et
    des paramètres de convolution (dimension du noyau et nombre de filtres en sortie)

    Output:
    model (keras.models) : modèle CNN-RNN (CNN-LSTM) à une couche de convolution, une couche LSTM, une couche dense cachée et une couche Dropout
    """
    if not isinstance(opti, str):
            raise Exception(
                " opti must be a string which represents optimizer : Adam or RMSprop")
    drop=0.3
    model = keras.Sequential()
    model.add(Conv1D(filters=filtre, kernel_size=kernel,
                         strides=1, padding="causal",
                         activation="relu",
                         input_shape=(look_back, len(columns)-1),
                         kernel_initializer='random_normal'))
    model.add(LSTM(ls, activation="tanh", return_sequences=False))
    model.add(Dropout(drop))
    model.add(Dense(1, activation="relu"))## sortie positive de dimension 1 

    if opti=="RMSprop":
        opt = keras.optimizers.RMSprop(learning_rate=learning_r,
        rho=0.9,
        momentum=0.0,
        epsilon=1e-07,
        centered=False,
        name="RMSprop")

    if opti=="Adam":
        opt =keras.optimizers.Adam(
        learning_rate=learning_r,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam")

    model.compile(optimizer =opt, loss = root_mean_squared_error,metrics = [keras.metrics.RootMeanSquaredError()])
    model.summary()

    return model          

In [ ]:
def create_model_ANN(opti='Adam',Den_1=8,Den_2=20,Den_3=10,learning_r=0.001):
    """
    Input:
    opti (string) : optimiseur à considérer pour l'entraînement du réseau de neurones. Les valeurs supportées sont : "RMSprop" et "Adam".
    Den_1 (int) : nombre de neurones dans la couche d'entrée Dense
    Den_2 (int) : nombre de neurones dans la 1ère couche cachée Dense
    Den_3 (int) : nombre de neurones dans la 2nde couche cachée Dense
    learning_r (float) : vitesse d'apprentissage du réseau

    Aim:
    Créer un modèle de réseau de neurones paramétrables par l'optimiseur, les nombres de neurones des couches Dense et la vitesse d'apprentissage.

    Output:
    model (keras.models) : modèle neuronal classique composée de couches Dense et Dropout
    """
    if not isinstance(opti, str):
            raise Exception(
                " opti must be a string which represents optimizer : Adam or RMSprop")
    if not isinstance(Den_1, int):
            raise Exception(
                " Den must be an int which represents number of neurons of dense")
    if not isinstance(Den_2, int):
            raise Exception(
                " Den must be an int which represents number of neurons of dense")
    if not isinstance(Den_3, int):
            raise Exception(
                " Den must be an int which represents number of neurons of dense")

    drop=0.3
    model = keras.Sequential()
    bias=True
    model.add(Dense(Den_1,use_bias=bias,input_shape=(look_back, len(columns)-1)))
    
    model.add(Dropout(drop))
    model.add(Flatten()) ## Attention "explosion" dimension car on flatten i.e. on repasse en dim 1 mais on multiplie la dim de l'espace latent de sortie précécent par le nombre de features = 16 
    # model.add(BatchNormalization())
    model.add(Dense(Den_2,use_bias=bias))
    model.add(Dropout(drop))
    # model.add(BatchNormalization())
    
    model.add(Dense(Den_3,use_bias=bias))
    model.add(Dropout(drop))
    # model.add(BatchNormalization())
    model.add(Dense(1,use_bias=bias))
    if opti=="RMSprop":
        opt = keras.optimizers.RMSprop(learning_rate=learning_r,
        rho=0.9,
        momentum=0.0,
        epsilon=1e-07,
        centered=False,
        name="RMSprop")## centered peut-être à mettre en True 

    if opti=="Adam":
        opt =keras.optimizers.Adam(
        learning_rate=learning_r,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam")

    model.compile(optimizer = opt, loss = root_mean_squared_error,metrics = [keras.metrics.RootMeanSquaredError()])
    model.summary()
    return model

In [ ]:
def get_error(models):
    """
    Input:
    models (list de keras.models) : liste de modèles issus d'une validation croisée dont on veut le        score RMSE moyen (sur le jeu de test)

    Aim:
    Retourner le score RMSE d'une liste de modèles entraînés par cross-validation

    Output:
    mean_RMSE = np.mean(rms_list) (float) : score RMSE moyen de la liste de modèles
    """
    if not isinstance(models, list):
            raise Exception(
                " models must be a list which represents cross validated models")
    rms_list=[]
    for p in range(len(models)):
        y_pred=[]
        for i in range(len(list_X_test)):
            pred=models[p].predict(list_X_test[i])
            for j in range(len(pred)):
                y_pred.append(pred[j])
        rms = mean_squared_error(np.ravel(list_y_test), y_pred, squared=False)
        rms_list.append(rms)
    
    return np.mean(rms_list)

In [ ]:
def training(type_ANN,opti,batch=32,nb_epochs=150):
    """
    Input:
    type_ANN (string) : type d'Artifical Neural Network (ANN) que l'on souhaite entraîner
    opti (string) : nom de l'optimiseur à utiliser pour l'entraînement. Les valeurs supportées sont : "RMSprop" et "Adam".
    batch (int) : taille des batchs à donner en entrée du réseau de neurones
    nb_epochs : longueur (en itérations) des époques d'entraînement du réseau

    Aim:
    Entraîner le modèle spécifié par validation croisée en mettant en place un Early Stopping (arrêt de l'entraînement si des signes d'overfitting apparaissent).
    Stocker les valeurs des erreurs d'entraînement et de test pour représentation.

    Output:
    loss (list de float) : perte RMSE du modèle sur le jeu d'entraînement au fil des époques
    loss_val (list de float) : perte RMSE du modèle sur le jeu de validation au fil des époques
    models (list de keras.models) : modèles entraînés par validation croisée
    erreur (float) : perte RMSE moyenne des modèles sur le jeu de test
    """
    if not isinstance(type_ANN, str):
            raise Exception(
                " type_ANN must be a string which represents the type of model you want to train : ANN, RNN, CNN_RNN")
    if not isinstance(opti, str):
            raise Exception(
                " opti must be a string which represents optimizer : Adam or RMSprop")
    if not isinstance(batch, int):
            raise Exception(
                " Batch must be an int which represents batch_size")
    if not isinstance(nb_epochs, int):
            raise Exception(
                " nb_epochs must be an int which represents number of epochs")


    loss,loss_val,weights,models=[],[],[],[]

    for i in range(len(list_X_train)):
        X_train=list_X_train[i]
        y_train=list_y_train[i]
        X_test=list_X_test[i]
        y_test=list_y_test[i]
        print("Cross Val no : ", i )
        if type_ANN=="ANN":
            model=create_model_ANN(opti)
        if type_ANN=="LSTM":
            model=create_model_LSTM(opti)
        if type_ANN=="CNN_RNN":
            model=create_model_CNN_RNN(opti)
        
        callback = EarlyStopping(monitor='val_root_mean_squared_error', patience=10)
        history=model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = batch, epochs = nb_epochs,verbose=False,callbacks=[callback])
        models.append(model)
        print(history.history.keys())
        loss.append(history.history["loss"])
        print("stopped at " ,len(history.history["loss"])," epochs")
        loss_val.append(history.history['val_root_mean_squared_error'])
    
        weights.append(model.get_weights())
    erreur=get_error(models)
    return loss,loss_val,models,erreur

In [ ]:
def plot_pred(models,titre):

    """
    Input:
    models (list de keras.model) : liste de modèles entraînés par validation croisée (tous du même type)
    titre (string) : titre du graphe

    Aim:
    Représenter les prédictions des modèles en fonction des vraies valeurs sous forme de scatter plot

    Output:
    None
    """
    colormap=plt.cm.gist_ncar
    colost=[colormap(i) for i in np.linspace(0,0.95,len(models)+1)]## len du nb de trucs à plot
    plt.figure()
    for p in range(len(models)):
        y_pred=[]
        for i in range(len(list_X_test)):
            pred=models[p].predict(list_X_test[i])
            for j in range(len(pred)):
                y_pred.append(pred[j])
        rms = mean_squared_error(np.ravel(list_y_test), y_pred, squared=False)
        plt.scatter(np.ravel(list_y_test),y_pred,color=colost[p],alpha=0.5,label=f"rmse : {round(rms,2)} $") 
    plt.plot(np.linspace(30000,60000),np.linspace(30000,60000),color="red") ### idendité i.e. cas idéal ou prédit = réel
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.title(titre)
    plt.legend()
    plt.grid()
    plt.show()
    return None

In [ ]:
def plot_loss(loss_list,loss_val_list,title):
    """
    Input:
    loss_list (list de float) : liste des pertes RMSE du modèle sur le jeu d'entraînement au fil des époques
    loss_val_list (list de float) : liste des pertes RMSE du modèle sur le jeu de validation au fil des époques
    titre (string) : titre du graphe

    Aim:
    Représenter les pertes des modèles en fonction des époques

    Output:
    None
"""
    for i in range(len(loss_list)):
        plt.figure()
        plt.plot(loss_list[i],color="orange",label="loss")
        plt.plot(loss_val_list[i],color="blue",label="val loss")
        plt.title(f'model {title} loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(loc='upper right')
        plt.grid()
        plt.show()

### 5.2 - Etude du modèle LSTM 

In [ ]:
param_distribs = {
 "opti":["Adam","RMSprop"],
 "ls": [25,35,50,60],
 "Den": [10,20,25,30],
 "learning_r": reciprocal(3e-4, 3e-2),
}
X_train=list_X_train[0]
y_train=list_y_train[0]
y_valid=list_y_test[0]
X_valid=list_X_test[0]
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(create_model_LSTM)
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,n_jobs=-1)# n_jobs pour exploiter tous les cpu 
rnd_search_cv.fit(X_train, y_train, epochs=25,
 validation_data=(X_valid, y_valid),
 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model_LSTM = rnd_search_cv.best_estimator_.model

On récupère alors les meilleurs hyperparamètres, que l'on va mettre comme par défault dans la fonction de création. On va ensuite chercher à regarder l'impact du batch_size. 

In [ ]:
for batch in [32,64,128]:
    loss,loss_val,models,err=training("LSTM","RMSprop",batch,500)
    print("RMSE erreur en $ USD",err)
    plot_loss(loss,loss_val,"LSTM ")
    plot_pred(models,"Prediction")

En défilant la barre de droite, on observe plusieurs diagrammes de perte. Pour la première série, on voit les pertes sur les jeux d'entraînement et de validation pour un modèle à couches Dense classique en fonction de itérations de validation croisée. Au fur et à mesure des itérations, l'erreur de validation decroît pour arriver au même niveau que l'erreur d'entraînement. Le graphe de type "scatter plot" suivant indique la distribution des points (valeur prédite, valeur réelle), la courbe rouge représentant la prédiction idéale. On note des RMSE élevés, le meilleur score étant à 9k$. Ces simulations ont été faites pour une taille de batch de 128. La même opération est repétée pour des tailles de batch de 32 et 64 (les graphes sont à la suite). Même si les valeurs de RMSE sont toutes mauvaises, on remarque qu'une taille de 64 peut être retenue pour la suite car donne les meilleurs résultats.

In [ ]:
X_train=list_X_train[-1]
y_train=list_y_train[-1]
y_valid=list_y_test[-1]
X_valid=list_X_test[-1]

callback = EarlyStopping(monitor='val_root_mean_squared_error', patience=10)
t1=time.time()
mode_lstm=create_model_LSTM()
mode_lstm.fit(X_train, y_train, validation_data = (X_valid, y_valid), batch_size = 128, epochs = 1000,verbose=False,callbacks=[callback])# mettre le meilleur opti 
t_lstm=time.time()-t1

In [ ]:
print("The LSTM model was trained {} s".format(t_lstm))

In [ ]:
plot_pred([mode_lstm],"Prédiction")

On observe que la prédiction du modèle LSTM + Dense est mauvaise. En effet, le réseau ne prédit que des valeurs entre 45k$ et 55k$. Il faut par conséquent construire des modèles différents pour améliorer la métrique RMSE. On va à ce titre explorer les modèles Dense classiques et les LSTM avec une couche de convolution (CNN-LSTM).

In [ ]:
mode_lstm.save(r"..\models\LSTM")

### 5.3 - Etude du modèle classique (couches Dense), dit ANN 

In [ ]:
param_distribs = {
 "opti":["Adam","RMSprop"],
 "Den_1": [4,6,8],
 "Den_2": [20,30,50],
 "Den_3": [5,10,15],
 "learning_r": reciprocal(3e-4, 3e-2),
}
X_train=list_X_train[0]
y_train=list_y_train[0]
y_valid=list_y_test[0]
X_valid=list_X_test[0]
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(create_model_ANN)
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,n_jobs=-1)
rnd_search_cv.fit(X_train, y_train, epochs=50,
 validation_data=(X_valid, y_valid),
 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model_LSTM = rnd_search_cv.best_estimator_.model

On récupère alors les meilleurs hyperparamètres, que l'on va mettre comme par défault dans la fonction de création. On va ensuite chercher à regarder l'impact du batch_size. 

In [ ]:
for batch in [32,64,128]:
    loss,loss_val,models,err=training("ANN","Adam",batch,500)## attention changer ADAM si c'est pas le meilleurs
    print("RMSE erreur en $ USD",err)
    plot_loss(loss,loss_val,"ANN ")
    plot_pred(models,"Prediction")

En défilant la barre de droite, on observe plusieurs diagrammes de perte. Pour la première série, on voit les pertes sur les jeux d'entraînement et de validation pour un modèle à couches Dense classique en fonction de itérations de validation croisée. Au fur et à mesure des itérations, l'erreur de validation decroît pour arriver au même niveau que l'erreur d'entraînement. Le graphe de type "scatter plot" suivant indique la distribution des points (valeur prédite, valeur réelle), la courbe rouge représentant la prédiction idéale. On note des RMSE élevés, le meilleur score étant à 9k$. Ces simulations ont été faites pour une taille de batch de 64. La même opération est repétée pour des tailles de batch de 32 et 128 (les graphes sont à la suite). Même si les valeurs de RMSE sont toutes mauvaises, on remarque qu'une taille de 64 peut être retenue pour la suite car donne les meilleurs résultats.

In [ ]:
X_train=list_X_train[-1]
y_train=list_y_train[-1]
y_valid=list_y_test[-1]
X_valid=list_X_test[-1]

callback = EarlyStopping(monitor='val_root_mean_squared_error', patience=10)
t2=time.time()
mode_ANN=create_model_ANN()
mode_ANN.fit(X_train, y_train, validation_data = (X_valid, y_valid), batch_size = 64, epochs = 1000,verbose=False,callbacks=[callback])# mettre le meilleur opti 
t_ann=time.time()-t2
plot_pred([mode_ANN],"Prédiction")

In [ ]:
print(f"the ANN model was trained in {t_ann} seconds")

In [ ]:
mode_ANN.save(r"..\models\ANN")

### 5.4 - Etude du modèle CNN-RNN (réseau convolutionnel récurrent, ou CNN/LSTM) 

In [ ]:
param_distribs = {
 "opti":["Adam","RMSprop"],
 "filtre": [2,3,4,5,6,7,8],
 "kernel": [3,4,5,6],
 "ls":[3,4,5,6],
 "learning_r": reciprocal(3e-4, 3e-2),
}
X_train=list_X_train[0]
y_train=list_y_train[0]
y_valid=list_y_test[0]
X_valid=list_X_test[0]
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(create_model_CNN_RNN)
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,n_jobs=-1)
rnd_search_cv.fit(X_train, y_train, epochs=50,
 validation_data=(X_valid, y_valid),
 callbacks=[keras.callbacks.EarlyStopping(patience=10)])


In [ ]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model_CNN_RNN = rnd_search_cv.best_estimator_.model

On récupère alors les meilleurs hyperparamètres, que l'on va mettre comme par défault dans la fonction de création. On va ensuite chercher à regarder l'impact du batch_size. 

In [ ]:
for batch in [32,64,128]:
    loss,loss_val,models,err=training("CNN_RNN","RMSprop",batch,500)
    print("RMSE erreur en $ USD",err)
    plot_loss(loss,loss_val,"CNN_RNN ")
    plot_pred(models,"Prediction")

En défilant la barre de droite, on observe plusieurs diagrammes de perte. Pour la première série, on voit les pertes sur les jeux d'entraînement et de validation pour un modèle à couches Dense classique en fonction de itérations de validation croisée. Au fur et à mesure des itérations, l'erreur de validation decroît pour arriver au même niveau que l'erreur d'entraînement. Ici il y a clairement de l'underfitting on pourrait utiliser des méthodes pour augmenter artificellement le jeu de données. Le graphe de type "scatter plot" suivant indique la distribution des points (valeur prédite, valeur réelle), la courbe rouge représentant la prédiction idéale. On note des RMSE élevés, le meilleur score étant à 9k$. Ces simulations ont été faites pour une taille de batch de 64. La même opération est repétée pour des tailles de batch de 32 et 128 (les graphes sont à la suite). Même si les valeurs de RMSE sont toutes mauvaises, on remarque qu'une taille de 64 peut être retenue pour la suite car donne les meilleurs résultats.

In [ ]:
X_train=list_X_train[-1]
y_train=list_y_train[-1]
y_valid=list_y_test[-1]
X_valid=list_X_test[-1]

callback = EarlyStopping(monitor='val_root_mean_squared_error', patience=10)
t2=time.time()
mode_CNN_RNN=create_model_CNN_RNN()
mode_CNN_RNN.fit(X_train, y_train, validation_data = (X_valid, y_valid), batch_size =64 , epochs = 1000,verbose=False,callbacks=[callback])# mettre le meilleur opti 
t_cnn=time.time()-t2
plot_pred([mode_CNN_RNN],"Prédiction")

In [ ]:
print(f"the ANN model was trained in {t_cnn} seconds")

In [ ]:
mode_CNN_RNN.save(r"..\models\RNN_CNN")

## 6 - Conclusion 
We have tried out to select some models but with more time we are convinced that we could have builded much more efficient models. 
To sum up our results : 

| Model | RMSE | Time | Numbers of params|
|:-:|:-:|:-:|:-:|
| LSTM | 8854 $ USD |17.82 s  | 7 511 |
| ANN Dense| 10616 $ USD  |4.18 s | 3 569 |
| CNN + LSTM  | 42 172 $ USD | 197.13 s | 324 |

We can say that was definitly a difficult subject with the fact that the end of the time serie did not happend in the past. 


## 7 - Sources : 
* [Batch size issue ] (https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/?fbclid=IwAR3zjCnpr8tC4nYN4q2m1LXbyteFECLqfLgBupzsz5x_FWdQN76Hu3Z7Hck)
* [feature engineering random forest] (https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)
* [RNN cheatsheet](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks)
* [ RmsProp basics ](https://towardsdatascience.com/a-look-at-gradient-descent-and-rmsprop-optimizers-f77d483ef08b)
* [ RMSprop keras](https://keras.io/api/optimizers/rmsprop/)
* [SGD](https://scikit-learn.org/stable/modules/sgd.html)
* [Loss functions & time series] (http://www.faculty.ucr.edu/~taelee/paper/lossfunctions.pdf)
* [Hand on machine learning with scikit-learn, keras & tensorflow]

## 8 - Annexe 

## 8.1 - Etude du modèle LSTM Simple

In [ ]:
number_features = len(columns) - 1 # nombre de variables explicatives
batch_size = 10

In [ ]:
# on construit un modèle de réseau de neurones récurrent de type LSTM (Long Short-Term Memory)
def create_simple_LSTM_model():
    # Build the model
    model = keras.Sequential()
    model.add(LSTM(1,
                batch_input_shape = (None, look_back, number_features)))
    model.add(Dense(1))
    model.compile(optimizer = "adam", loss = 'mse')
    return model

In [ ]:
model = create_simple_LSTM_model()
#loss = [] # valeur de la fonction de perte sur le jeu de test
#loss_val = [] # valeur de la fonction de perte sur le jeu d'entraînement
i = 1
X_train = list_X_train[i]
y_train = list_y_train[i]    
X_test = list_X_test[i]
y_test = list_y_test[i]
history = model.fit(X_train,y_train,validation_data = (X_test, y_test), batch_size = batch_size, epochs = 10)

In [ ]:
epochs = np.linspace(1, 10, 10)
losses = history.history["loss"] # perte du jeu d'entraînement
val_losses = history.history["val_loss"] # perte du jeu de test
plt.plot(epochs, losses, "xb")
plt.plot(epochs, val_losses, "xr")
plt.title("Perte en fonction des époques")
plt.xlabel("Epoques")
plt.ylabel("Perte (Mean Squared Error)")
plt.legend(["Entraînement", "Test"])
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred[:, -1]
print(y_pred, y_pred.shape)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("La RMSE du modèle est :", rmse)

## 8.2 - Etude du modèle LSTM simple avec couche Dropout
On ajoute un "dropout" au réseau LSTM simple. Ce paramètre permet de fixer une certaine proportion des poids pour une itération (ils ne seront pas modifiés par back-propagation).

In [ ]:
# on construit un modèle de réseau de neurones récurrent de type LSTM (Long Short-Term Memory)
def create_dropout_LSTM_model(dropout):
    # Build the model
    model = keras.Sequential()
    model.add(LSTM(1,
                batch_input_shape = (None, look_back, number_features),
                dropout = dropout))
    model.add(Dense(1))
    model.compile(optimizer = "adam", loss = 'mse')
    return model

In [ ]:
i=1
X_train = list_X_train[i]
y_train = list_y_train[i]    
X_test = list_X_test[i]
y_test = list_y_test[i]
history = model.fit(X_train,y_train,
                validation_data = (X_test, y_test),
                batch_size = batch_size,
                epochs = 300)

In [ ]:
epochs = np.linspace(1, 300, 300)
losses = history.history["loss"] # perte du jeu d'entraînement
val_losses = history.history["val_loss"] # perte du jeu de test
plt.plot(epochs, losses, "xb")
plt.plot(epochs, val_losses, "xr")
plt.title("Perte en fonction des époques")
plt.xlabel("Epoques")
plt.ylabel("Perte (Mean Squared Error)")
plt.legend(["Entraînement", "Test"])
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred[:, -1]
print(y_pred, y_pred.shape)

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot(np.linspace(np.min(y_test),np.max(y_test), len(y_pred)),
        np.linspace(np.min(y_test),np.max(y_test), len(y_pred)),
        color = "r")
plt.title("Prédictions du réseau LSTM raffinné avec Dropout (1 couche)")
plt.xlabel("Valeurs réelles")
plt.ylabel("Valeurs prédites")
plt.show()

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("La RMSE du modèle est :", rmse)

On remarque que les premiers points prédits sont proches des valeurs réelles mais que le modèle ne parvient pas à prédire les hautes valeurs. Peut-on augmenter le dropout pour obtenir de meilleurs résultats ?

In [ ]:
model = create_dropout_LSTM_model(dropout = 0.4)
history = model.fit(X_train,
                    y_train,
                    validation_data = (X_test, y_test),
                    batch_size = batch_size,
                    epochs = 200)

In [ ]:
epochs = np.linspace(1, 200, 200)
losses = history.history["loss"] # perte du jeu d'entraînement
val_losses = history.history["val_loss"] # perte du jeu de test
plt.plot(epochs, losses, "xb")
plt.plot(epochs, val_losses, "xr")
plt.title("Perte en fonction des époques")
plt.xlabel("Epoques")
plt.ylabel("Perte (Mean Squared Error)")
plt.legend(["Entraînement", "Test"])
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred[:, -1, 0]
print(y_pred, y_pred.shape)

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot(np.linspace(np.min(y_test),np.max(y_test), len(y_pred)),
        np.linspace(np.min(y_test),np.max(y_test), len(y_pred)),
        color = "r")
plt.title("Prédictions du réseau LSTM raffinné avec Dropout (1 couche)")
plt.xlabel("Valeurs réelles")
plt.ylabel("Valeurs prédites")
plt.show()

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("La RMSE du modèle est :", rmse)